https://pythonprogramming.net/cryptocurrency-recurrent-neural-network-deep-learning-python-tensorflow-keras/?completed=/recurrent-neural-network-deep-learning-python-tensorflow-keras/

In [7]:
import pandas as pd
import os
from sklearn import preprocessing
from collections import deque
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"
import random
import numpy as np
def preprocess_df(df):
    df = df.drop('future', 1)

    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])

    random.shuffle(sequential_data)
    buys = []
    sells = []
    return sequential_data


def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"F:/Coding/crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
    
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)
main_df["target"] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))
# main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10)

times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

data = preprocess_df(main_df)




C:\Users\Marinus\AppData\Local\Temp\ipykernel_9384\3864133.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('future', 1)


[[array([[-2.51268250e-03, -7.89185022e-02, -5.40429356e-01,
          -1.17089656e-01,  4.60111811e-03, -4.78379997e-02,
           1.10617583e-02, -6.21709800e-03],
         [-2.29991651e-01,  3.40994132e-03,  4.04838613e-03,
          -1.18306852e-01,  4.60111811e-03, -4.53773491e-02,
           2.57552343e-01, -5.88380913e-03],
         [-8.30083053e-04, -7.40386327e-02,  4.04838613e-03,
          -7.29966458e-02,  2.12734814e-02, -3.29002556e-02,
           2.65415198e-01, -6.19975454e-03],
         [ 1.16298646e-01, -4.96264158e-02,  4.04838613e-03,
          -1.08241408e-01,  4.60111811e-03, -4.95302835e-02,
           4.55951014e-01, -5.89833729e-03],
         [ 9.36774676e-04, -8.20524538e-02,  4.04838613e-03,
          -4.85159270e-02,  4.60111811e-03, -3.29156249e-02,
           5.27186135e-01, -6.20610912e-03],
         [ 1.09471014e-01, -6.71763153e-02,  3.93210100e-01,
          -1.07576761e-01,  4.60111811e-03, -1.34541982e-02,
           3.10232629e-03, -6.10331066e-03]

In [11]:
data[3][0][0]

array([ 0.11629865, -0.04962642,  0.00404839, -0.10824141,  0.00460112,
       -0.04953028,  0.45595101, -0.00589834])

error: Library not initialized